In [1]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performer as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import glob

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
'''
def solve_cudnn_error():
    import tensorflow as tf
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
            
solve_cudnn_error()
'''

'\ndef solve_cudnn_error():\n    import tensorflow as tf\n    gpus = tf.config.experimental.list_physical_devices(\'GPU\')\n    if gpus:\n        try:\n            # Currently, memory growth needs to be the same across GPUs\n            for gpu in gpus:\n                tf.config.experimental.set_memory_growth(gpu, True)\n            logical_gpus = tf.config.experimental.list_logical_devices(\'GPU\')\n            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n        except RuntimeError as e:\n            # Memory growth must be set before GPUs have been initialized\n            print(e)\n            \nsolve_cudnn_error()\n'

In [3]:
import csv
def readCSV(file_name):
    errlist=[]
    LBlist=[]
    with open(file_name, newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary

        rows = csv.DictReader(csvfile)
        for row in rows: 
            RL=list(row.values())
            #print("RL[0]: ", type(RL[0]), "RL[1]: ", type(RL[1]))
            RL[1:] = list(map(int, RL[1:]))
            errs=RL[1:37]
            LB=RL[37:]
            errlist.append(errs)
            LBlist.append(LB)
    return errlist,LBlist


In [4]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [5]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [6]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [7]:
def parseSentence(x):	
    tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        #print(ord(x[i]))
        if (ord(x[i])>255):
            inp="U"
        else:
            inp="E"

        if state=="START":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])

        elif state=="ASCII":
            if inp=="E":
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="UNICODE":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])
    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
    return tokens

In [8]:
def readcode(fname):
    with open(fname,encoding = 'utf-8') as f:
        data = f.read()
        return data

In [9]:
from matplotlib import pyplot as plt
def plotTrainingLoss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

In [10]:
def plotTrainingErrorLineAcc(history):
    
    #======================accuracy======================
    acc = []
    #get all sample name
    names = []
    for sample in range(360):
        #get NN name
        name = "LNout" + str(sample) + "_accuracy"
        names.append(name)
    #get each epoch avg value
    for epoch in range(len(history.history[names[0]])): #get sample length
        one_epoch_sum = 0.0
        #print("epoch loop: ", epoch)
        for name in (names):
            one_epoch_sum += history.history[name][epoch]
        one_epoch_avg = one_epoch_sum / len(names) #get one epoch avg
        acc.append(one_epoch_avg) #return to acc
    plt.plot(acc)
    
    #======================validation accuary======================
    val_acc = []
    #get all sample name
    val_names = []
    for val_sample in range(360):
        #get NN name
        val_name = "val_LNout" + str(val_sample) +"_accuracy"
        val_names.append(val_name)
        
    #get each epoch avg value
    for epoch in range(len(history.history[names[0]])): #get sample length
        val_one_epoch_sum = 0.0
        for name in (val_names):
            val_one_epoch_sum += history.history[name][epoch]
        val_one_epoch_avg = val_one_epoch_sum / len(val_names) #get one epoch avg
        val_acc.append(val_one_epoch_avg) #return to acc
    plt.plot(val_acc)
    
    plt.title('model acc')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.show()

In [11]:
def plotTrainingErrorTypeAcc(history):
    plt.plot(history.history['error_feed_forward_output1_accuracy'])
    plt.plot(history.history['val_error_feed_forward_output1_accuracy'])
    plt.title('model error_feed_forward_output1_accuracy')
    plt.ylabel('error_feed_forward_output1_accuracy')
    plt.xlabel('epoch')
    plt.show()

In [12]:
import os
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [13]:
#save model for training
class TestTranslate(unittest.TestCase):
        
    def __init__(self):
        self.source_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<CR>': 5,
        }
        
    @staticmethod
    def _build_token_dict(token_dict, token_list):
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
    
    def test_translate(self):
        #print("i am here: " )
        #source_file=[]
        #Set Para
        max_javaline_length = 160 #Max number of lines
        #set path
        Output_Path = "Trianing\InputCSV\Split-500-reduce"
        Input_Path = "Trianing\InputTxt\Split-500-reduce"
        model_for_training_org_path = "Model-for-training-org\Split-500-reduce"
        model_for_training_path = "Model-for-training\Split-500-reduce"
        Trained_model_Path = "Trained_models\Split-500-reduce"
        #get all txt file in input path
        in_path = (glob.glob(Input_Path + "/**/*.txt"))
        #print("in_path: ", in_path)
        #cases = listdir_fullpath(Input_Path)
        source_max_len = 0
        target_max_len = 0
        token_num = 0
        all_sample_num = 16644 #all sample number
        block_num = 16644 #sample num e.g 10000 sample have 10*1000
        
        #''' <----dust switch, if you need to create npy model for traing open this
        self.sl = 0
        
        import math
        for loop in range(0, math.ceil(all_sample_num/block_num)): #new version 
        #for loop in range(0, round(336/block_num)): #old version
            print("First loop: ", loop)
            source_tokens = []
            target_errors = []
            target_LB = []
            if(all_sample_num % block_num == 0):
                dirs = block_num
            else:
                dirs = block_num if loop < all_sample_num // block_num else all_sample_num % block_num
            Input_fullpath = []
            Output_fullpath = []
            #print("dirs: ", dirs)
            for i in range(dirs):
                Input_fullpath.append(in_path[loop*block_num + i])                    
            for f in Input_fullpath:
                if isfile(f):
                    source_tokens.append(parseSentence(readcode(f)))
                    #print("source_tokens length: ", len(source_tokens))
                #if len(source_tokens)>max_files: break
            #get csv file     
            out_path = Output_Path + "/" + "test" + str(loop)+".csv"
            Output_fullpath = glob.glob(out_path)
            
            for f in Output_fullpath:
                if isfile(f):
                    err,lb = readCSV(f)
                    target_errors.append(err)
                    target_LB.append(lb)
                #if len(source_tokens)>max_files: break
            dd = np.asarray(target_errors)
            target_errors = target_errors[0]  
            target_LB = target_LB[0]     
            
            # Generate dictionaries
            self._build_token_dict(self.source_token_dict, source_tokens)
            
            # Add special tokens
            encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
            
            #output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens] 
            
            self.sl = max(list(map(len, encode_tokens))+[self.sl])
            source_max_len = self.sl

        #padding here
        print("source_max_len:", source_max_len)
        for loop in range(0, math.ceil(all_sample_num/block_num)): #new version
        #for loop in range(0, round(336/block_num)): #old version
            print("Second loop: ", loop)
            source_tokens = []
            target_errors = []
            target_LB = []
            if(all_sample_num % block_num == 0):
                dirs = block_num
            else:
                dirs = block_num if loop < all_sample_num // block_num else all_sample_num % block_num
            Input_fullpath = []
            Output_fullpath = []
            for i in range(dirs):
                Input_fullpath.append(in_path[loop*block_num + i])
            for f in Input_fullpath:
                if isfile(f):
                    source_tokens.append(parseSentence(readcode(f)))

            out_path = Output_Path + "/" + "test"+ str(loop)+ ".csv"
            Output_fullpath = glob.glob(out_path)
            for f in Output_fullpath:
                if isfile(f):
                    err, lb = readCSV(f)
                    target_errors.append(err)
                    target_LB.append(lb)
            #if len(source_tokens)>max_files: break
            dd = np.asarray(target_errors)
            #print("AAAA: ", dd.shape)
            #print("aaaa: " , type(target_errors[0][0]))
            target_errors = target_errors[0]  
            target_LB = target_LB[0]     
            #print("source_token legth: " , len(source_tokens))
            #print("YYYY: " , type(target_errors[0][0]))
            #print("ZZZZ: " , len(target_LB))
            #print("ZZZZ len(target_LB[0]): " , len(target_LB[0]))
            #print("XXXX2: " , len(source_tokens))
            # Generate dictionaries        
            self._build_token_dict(self.source_token_dict, source_tokens)

            # Add special tokens
            encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
            #print("encode_tokens1: ", encode_tokens)
            encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
            #print("encode_tokens2: ", encode_tokens)
            encode_input = [list(map(lambda x: self.source_token_dict[x], tokens)) for tokens in encode_tokens]
            #print("encode_input1: ", encode_input)
            token_num = len(self.source_token_dict)
            #print("token num: ", token_num)
            #print(token_num)
            #print(type(token_num))
            #define save path and save dict  
            dict_name = "source_token_dict.pickle"
            #save for org
            saveDictionary(self.source_token_dict, model_for_training_org_path + "/" + dict_name)
            #save for training
            saveDictionary(self.source_token_dict, model_for_training_path + "/" + dict_name)
            #print("x.shape", np.asarray(encode_input).shape)  #x.shape (2,  9)
            
            #x=[np.array(encode_input * 1)]
            #y=[np.array(target_errors * 1),np.array(target_LB * 1)]

            #print("x.shape", np.asarray(x).shape)  #x.shape (2, 2048, 9)

            ####  Split the data set into train and test_model
            x = np.asarray(encode_input)
            y = list(zip(np.asarray(target_errors), np.asarray(target_LB)))

            #print("x.shape: ", x.shape)
            #print("y length: ", len(y))
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=42)
            
            '''
            y_train = list(zip(*y_train))  
            y_train[0] = np.asarray(y_train[0])
            y_train[1] = np.asarray(y_train[1])

            y_train[1] = to_categorical(y_train[1], num_classes=max_javaline_length) #將類別向量轉換為二進制矩陣
            #y_train = list(zip(y_train[0], y_train[1]))

            #print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX:" , y_train[0].shape)
            #print("y_train[1].shape", y_train[1].shape)
            #print("???????????: y_train.length ", len(y_train))
            #print("???????????: y_train.[1] type ", type(y_train[1]))
            y_train[1] = np.split(y_train[1], indices_or_sections=len(target_LB[0]), axis=1)
            y_train[1] = [np.squeeze(elm, axis = 1) for elm in y_train[1]]
            #print("after change->len(y_train[1].shape)", len(y_train[1]) )
            '''
            
            y_test = list(zip(*y_test))
            y_test[0] = np.asarray(y_test[0])
            y_test[1] = np.asarray(y_test[1])
            y_test[1] = to_categorical(y_test[1], num_classes = max_javaline_length) 
            #y_test = list(zip(y_test[0], y_test[1])) 
            y_test[1] = np.split(y_test[1], indices_or_sections=len(target_LB[0]), axis=1) 
            y_test[1] = [np.squeeze(elm, axis = 1) for elm in y_test[1]]           
            
            #=============================#
            x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.1, random_state=42)
            print("Split x_train shape: ", (x_train).shape)
            print("Split x_validation shape: ", (x_validation).shape)
            
            y_train = list(zip(*y_train))  
            y_train[0] = np.asarray(y_train[0])
            y_train[1] = np.asarray(y_train[1])

            y_train[1] = to_categorical(y_train[1], num_classes=max_javaline_length) #將類別向量轉換為二進制矩陣
            #y_train = list(zip(y_train[0], y_train[1]))

            #print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX:" , y_train[0].shape)
            #print("y_train[1].shape", y_train[1].shape)
            #print("???????????: y_train.length ", len(y_train))
            #print("???????????: y_train.[1] type ", type(y_train[1]))
            y_train[1] = np.split(y_train[1], indices_or_sections=len(target_LB[0]), axis=1)
            y_train[1] = [np.squeeze(elm, axis = 1) for elm in y_train[1]]
            #print("after change->len(y_train[1].shape)", len(y_train[1]) )

            y_validation = list(zip(*y_validation))
            y_validation[0] = np.asarray(y_validation[0])
            y_validation[1] = np.asarray(y_validation[1])
            y_validation[1] = to_categorical(y_validation[1], num_classes = max_javaline_length) 
            #y_test = list(zip(y_test[0], y_test[1])) 
            y_validation[1] = np.split(y_validation[1], indices_or_sections=len(target_LB[0]), axis=1) 
            y_validation[1] = [np.squeeze(elm, axis = 1) for elm in y_validation[1]]           
            
            
            #save org model for training 
            saveTestTrainData(model_for_training_org_path + "/" + "x_train_" + str(loop) + ".npy", x_train)
            saveTestTrainData(model_for_training_org_path + "/" + "y_train[0]_" + str(loop) + ".npy", y_train[0])
            saveTestTrainData(model_for_training_org_path + "/" + "y_train[1]_" + str(loop) + ".npy", y_train[1])
            
            saveTestTrainData(model_for_training_org_path + "/" + "x_test_" + str(loop) + ".npy", x_test)
            saveTestTrainData(model_for_training_org_path + "/" + "y_test[0]_" + str(loop) + ".npy", y_test[0])
            saveTestTrainData(model_for_training_org_path + "/" + "y_test[1]_" + str(loop) + ".npy", y_test[1])
            
            #=========================================
            saveTestTrainData(model_for_training_org_path + "/" + "x_validation_" + str(loop) + ".npy", x_validation)
            saveTestTrainData(model_for_training_org_path + "/" + "y_validation[0]_" + str(loop) + ".npy", y_validation[0])
            saveTestTrainData(model_for_training_org_path + "/" + "y_validation[1]_" + str(loop) + ".npy", y_validation[1])
            
            #transform x_train
            #Set model para
            training_source_max_len = 300 #for replace source_max_len
            print("org x_train shape: ", (x_train).shape)
            print("org x_test shape: ", (x_test).shape)
            #reshape x_train
            new_x_train = []
            for i in range(len(x_train)):
                #print("i: ", i)
                d_two = []
                for k in range(training_source_max_len): #set length
                    #print("k: ", k)
                    d_two.append(x_train[i][k])
                #switch to np array
                d_two = np.asarray(d_two)
                #give np array
                new_x_train.append(d_two)
            #switch to np array
            x_train = np.asarray(new_x_train)
            #save split model for training
            
            #reshape x_test
            new_x_test = []
            for i in range(len(x_test)):
                #print("i: ", i)
                d_two = []
                for k in range(training_source_max_len): #set length
                    #print("k: ", k)
                    d_two.append(x_test[i][k])
                #switch to np array
                d_two = np.asarray(d_two)
                #give np array
                new_x_test.append(d_two)
            #switch to np array
            x_test = np.asarray(new_x_test)
            
            #reshape x_validation
            new_x_validation = []
            for i in range(len(x_validation)):
                #print("i: ", i)
                d_two = []
                for k in range(training_source_max_len): #set length
                    #print("k: ", k)
                    d_two.append(x_validation[i][k])
                #switch to np array
                d_two = np.asarray(d_two)
                #give np array
                new_x_validation.append(d_two)
            #switch to np array
            x_validation = np.asarray(new_x_validation)
            
            #save split model for training
            saveTestTrainData(model_for_training_path + "/" + "x_train_" + str(loop) + ".npy", x_train)
            saveTestTrainData(model_for_training_path + "/" + "y_train[0]_" + str(loop) + ".npy", y_train[0])
            saveTestTrainData(model_for_training_path + "/" + "y_train[1]_" + str(loop) + ".npy", y_train[1])
            
            saveTestTrainData(model_for_training_path + "/" + "x_test_" + str(loop) + ".npy", x_test)
            saveTestTrainData(model_for_training_path + "/" + "y_test[0]_" + str(loop) + ".npy", y_test[0])
            saveTestTrainData(model_for_training_path + "/" + "y_test[1]_" + str(loop) + ".npy", y_test[1])
            
            #=====================================
            saveTestTrainData(model_for_training_path + "/" + "x_validation_" + str(loop) + ".npy", x_validation)
            saveTestTrainData(model_for_training_path + "/" + "y_validation[0]_" + str(loop) + ".npy", y_validation[0])
            saveTestTrainData(model_for_training_path + "/" + "y_validation[1]_" + str(loop) + ".npy", y_validation[1])
        print("Training model save successful...")    
        #'''
        
        
        #start training
        import DataGeneratorTrain as DGTrain
        import DataGeneratorValidation as DGValidation
        import DataBuffer as db
        from random import randrange
        #Set driver path
        source_max_len = 1200 #set max len  #default : 2889
        line_block_num = 360 #lbNum
        source_token_dict_name = "source_token_dict.pickle"
        #load source_token_dict
        source_token_dict = loadDictionary(model_for_training_path + "/" + source_token_dict_name)
        #Set model para    
        model = tfr.get_model(max_input_len=(source_max_len),
                              max_javaline_length=160,
                              errNum=36,
                              lbNum=line_block_num, #lbNum=len(target_LB[0]), #160
                              token_num=len(source_token_dict),
                              embed_dim=256, #32, try 32 or 64
                              encoder_num=4, #2 max = 6
                              head_num=4,#4
                              hidden_dim=128, #128
                              dropout_rate=0.05 #0.05
                             )
        #Set losses
        losses = {"error_feed_forward_output1": "binary_crossentropy"}
        #error type weight
        lossWeights = {"error_feed_forward_output1": 1.0}
        
        #metrics = {"error_feed_forward_output1": tf.keras.metrics.Accuracy()}
        
        #error line weight
        for i in range(line_block_num):
            name = "LNout" + str(i)
            losses[name] = "categorical_crossentropy"
            lossWeights[name] = 100 #error_feed_forward_output2[] weight
            #metrics[name] = tf.keras.metrics.CategoricalAccuracy()
        
        
        #set complie para
        model.compile(optimizer=Adam(learning_rate=0.000001), loss=losses, loss_weights=lossWeights, metrics=["accuracy"])
        
        #for output
        #for x
        input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": 13481,
            "data_type": int,
            "block_size": 13481 
            }
        
        #for input
        #for y
        output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [13481, 13481],
            "data_type": [int, int],
            "block_size": [13481, 13481] 
            }
        
        #===========================================
        #for output
        #for x
        validation_input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": 1498,
            "data_type": int,
            "block_size": 1498 
            }
        
        #for input
        #for y
        validation_output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [1498, 1498],
            "data_type": [int, int],
            "block_size": [1498, 1498] 
            }
        
        
        #Create Generators
        print("Creating training generator...")
        training_generator = DGTrain.DataGeneratorTrain(input_buffer_params,
                                                  output_buffer_params,
                                                  [list(range(13481)), list(range(13481))] 
                                                )
        #Create Generators
        print("Creating validation generator...")
        validation_generator = DGValidation.DataGeneratorValidation(validation_input_buffer_params,
                                                  validation_output_buffer_params,
                                                  [list(range(1498)), list(range(1498))] 
                                                )
        
        
        #Start training
        print("Strat training...")
        
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = Trained_model_Path + "/" + "checkpoint_model.h5",
                                                                       monitor = "val_loss",
                                                                       mode = "min",
                                                                       save_best_only = True
                                                                      )
        
        history = model.fit_generator(generator = training_generator,
                                      epochs = 2000, #100 200 500 3000
                                      verbose = 3, #set visibility
                                      validation_data = validation_generator,
                                      callbacks = [model_checkpoint_callback],
                                     )
        
        
        print("Model training completed...")
        #save history
        print("Saving history...")
        saveDictionary(history.history, Trained_model_Path + "/" + "model_history")
        print("History saving completed...")
        
        #save model
        print("Saving model...")
        model.save(Trained_model_Path + "/" + "test_model1.h5")
        print("Model saving completed...")
        
        #print("history.history.keys: ", history.history.keys())
        #show loss grapgh
        plotTrainingLoss(history)
        plotTrainingErrorTypeAcc(history)
        plotTrainingErrorLineAcc(history)
        
        '''
        model, source_token_dict = load("test_model1.h5")
        
        out1, out2 = tfr.decode(
                model,
                #encode_input,
                x_test_loaded,max_len=source_max_len
            )
        #'''
        
    def getsource_max_lan(self):
        return self.sl
    

In [14]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [15]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [16]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [17]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [18]:
def load(model_name):
        import sys
        #sys.path.append('/content/drive/MyDrive/Final_Edition_include_model')
        sys.path.append("Performer\Performer_local_V3\keras_layer_normalization")
        sys.path.append("Performer\Performer_local_V3\keras_performer")
        sys.path.append("Performer\Performer_local_V3\keras_position_wise_feed_forward")
        sys.path.append("Performer\Performer_local_V3\tensorflow_fast_attention")

        from keras_performer import performer
        from tensorflow import keras
        from keras_embed_sim import EmbeddingRet, EmbeddingSim
        from keras_pos_embd import TrigPosEmbedding
        from tensorflow_fast_attention.fast_attention import  Attention, SelfAttention
        from keras_position_wise_feed_forward.feed_forward import FeedForward  

        co = performer.get_custom_objects()

        model = keras.models.load_model(model_name, custom_objects= co)
        source_token_dict = loadDictionary("source_token_dict.pickle")
       # t = loadDictionary(target_token_dict, 'target_token_dict.pickle')
       # t_inv = loadDictionary(target_token_dict_inv, 'target_token_dict_inv.pickle')
        return model, source_token_dict,

In [ ]:
x=TestTranslate()
x.test_translate()

First loop:  0


In [ ]:
#set model para
model_name = "test_model1.h5"
x_test_model = "x_test_500.npy" 
y_test_mdodel1 = "y_test0_500.npy" 
y_test_mdodel2 = "y_test1_500.npy" 

In [ ]:
#load model
def loadmodel(model_name, x_test_model, y_test_mdodel1, y_test_mdodel2):
    import numpy as np
    #load model and dic ps. dic is not use
    model, source_token_dict = load(model_name)
    #print(model.summary())
    #load
    
    '''
    Para:
        x_test_loaded  : answer model        (include type and lineblock)
        y_test_loaded_0: predict model       (type)
        y_test_loaded_1: predict model       (lineblock)
        out1           : answer model output (type) 
        out2           : answer model output (lineblock)
    '''
    
    x_test_loaded = loadTestTrainData("test_models/" + x_test_model) 
    y_test_loaded_0 = loadTestTrainData("test_models/" + y_test_mdodel1)
    y_test_loaded_1 = loadTestTrainData("test_models/" + y_test_mdodel2) 
    
    ''' <-------dust switch
    print("y_test_loaded_1 shape: ", y_test_loaded_1.shape)
    print("x_test_loaded length: ", len(x_test_loaded))
    #'''
    
    out1, out2 = tfr.decode(model, x_test_loaded, max_len = x.getsource_max_lan())
    
    #==============show org result================
    ''' <-------dust switch
    print("y_test_loaded_0 shape: ", y_test_loaded_0.shape)
    print("y_test_loaded_1 shape: ", y_test_loaded_1.shape)
    print("y_test_loaded_0[0] result:", (y_test_loaded_0[0])) #Error_type #vs out1
    print("y_test_loaded_1[0][1] result:", (y_test_loaded_1[0][1])) #Line_Block #vs out2
    print("out1 shape: ", (out1).shape)#prob upper then 0.5
    print("out2[0] shape: ", (out2[0]).shape)#prob upper then 0.5
    print("out2 length: ", len(out2))
    print("out2[0] length: ", len(out2[0]))#prob lb
    #'''
    
    #=============================================
    test_ep = np.around(out1)
    test_lb = np.around(out2)
    ans_ep = np.around(y_test_loaded_0)
    ans_lb = np.around(y_test_loaded_1)
    #==============show toint result==============
    
    ''' <-------dust switch
    print(test_ep[1])
    print(test_lb[1])
    print(ans_ep[1])
    print([ans_lb[1]])
    #'''
    #=============================================
    return test_ep, ans_ep, test_lb, ans_lb

In [ ]:
def intersect(pre_errortype, ans_errortype):
    #print("inter :", pre_errortype)
    #print("inter: ", ans_errortype)
    #ans_errortype = float(ans_errortype)
    #ref = "https://www.796t.com/post/Mjc4am8=.html"
    inter = [pre_value for pre_value, ans_value in zip(pre_errortype, ans_errortype) if (pre_value == ans_value == 1)]
    inter_two = [pre_value for pre_value, ans_value in zip(pre_errortype, ans_errortype) if (pre_value == ans_value)]
    #print("sort inter: ", inter)
    return inter, inter_two

In [ ]:
#get new ans array contains only 1 in array 
def ans_typefilter(ans_errortype):
    #print("org ans type: ", ans_errortype)
    new_ans_errortype = [value for value in ans_errortype if value == 1]
    #print("new ans errortype: ", new_ans_errortype)
    return new_ans_errortype

In [ ]:
#get new pre array contains only 1 in array 
def pre_typefilter(pre_errortype):
    #print("org error type: ", pre_errortype)
    new_pre_errortype = [value for value in pre_errortype if value == 1]
    #print("new pre errortype: ", new_pre_errortype)
    return new_pre_errortype

In [ ]:
#calculate error type score
#Note:
    #Predict score:
        # score = inter of ans and pre divide by len of pre
    #Recall score:
        # score = inter of ans and pre divide by len of ans
    #Accuracy score:
        # score = inter_two of ans and pre divide by original len of pre
def errortype_score(pre_errortype, ans_errortype):
    #find pre length
    new_pre_errortype = pre_typefilter(pre_errortype) #make new array for score
    pre_length = len(new_pre_errortype) #get lenght from ans_type
    #find ans length
    new_ans_errortype = ans_typefilter(ans_errortype) #make new array for score
    ans_length = len(new_ans_errortype) #get lenght from ans_type
    #print("ans length: ", ans_length)
    inter, inter_two = intersect(pre_errortype, ans_errortype) #get intersection
    inter_length = len(inter)
    inter_two_length = len(inter_two)
    #print("inter_length: ", inter_length)
    #print("pre length: ", pre_length) 
    
    #calculate Predict score 
    if (inter_length == 0 and pre_length == 0):
        pre_score = 1
    elif (pre_length == 0):
        pre_score = 0
    else:
        pre_score = inter_length/pre_length 
    #print("predict score: ", pre_score) #show pre score
    
    #calculate Recall score
    if (inter_length == 0 and ans_length == 0):
        rec_score = 1
    elif (ans_length == 0):
        rec_score = 0
    else:
        rec_score = inter_length/ans_length
    #print("recall score: ", rec_score) #show ans score
    
    #calculate Accuarcy score
    acc_score = inter_two_length/len(pre_errortype)
    
    return pre_score, rec_score , acc_score#return float

In [ ]:
#show total perdict score and recall score
def errortype_totalscore(pre_errortype,ans_errortype):
    #initial para
    pre_total = 0.0
    rec_total = 0.0
    acc_total = 0.0
    #get each score then get total and avg score
    for i in range(len(pre_errortype)):
        #print("Sample: ", i)
        pre, rec, acc = errortype_score(pre_errortype[i], ans_errortype[i])
        pre_total = pre_total + pre
        rec_total = rec_total + rec
        acc_total = acc_total + acc
    print("pre_total: ", pre_total)
    print("rec_total: ", rec_total)
    print("acc_total: ", acc_total)
    pre_avg_score = pre_total/len(pre_errortype)
    rec_avg_socre = rec_total/len(ans_errortype)
    acc_avg_score = acc_total/len(ans_errortype)
    print("avg_pre: ", pre_avg_score)#pre_total/len(pre_errortype))
    print("avg_rec: ", rec_avg_socre)#rec_total/len(ans_errortype))
    print("avg_acc: ", acc_avg_score)#acc_total/len(ans_errortype))
    return pre_avg_score, rec_avg_socre, acc_avg_score

In [ ]:
def get_start_end_line(pre_begin_index, pre_end_index, ans_begin_index, ans_end_index):
    #Note:
            #use softmax: [0.....150] total == 1
            #begin: index[0]
            #end: index[0]
        
    #get [value] first and get [value] again to get value
    pre_begin_line = pre_begin_index[0]
    pre_begin_line = pre_begin_line[0]
    pre_end_line = pre_end_index[0]
    pre_end_line = pre_end_line[0]
    ans_begin_line = ans_begin_index[0]
    ans_begin_line = ans_begin_line[0]
    ans_end_line = ans_end_index[0]
    ans_end_line = ans_end_line[0]
    
    return pre_begin_line, pre_end_line, ans_begin_line, ans_end_line

In [ ]:
def line_adjustment(line):
    if(line > 1):
        line = line-1
        return line
    else:
        return line

In [ ]:
# get each pre and ans lineblocks and make them to pre begin/end ans begin/end array
def make_lineblock(begin, end):
    if (end < begin):
        block = [value for value in range(end, begin+1)]
        return block
    else:
        block = [value for value in range(begin, end+1)]
        return block

In [ ]:
# ref = "https://www.geeksforgeeks.org/python-intersection-two-lists/"
def lineblock_intersect(pre_block, ans_block):
    inter = [value for value in pre_block if value in ans_block]
    return inter

In [ ]:
# return index with int datatype 
def get_block_index(pre_begins, pre_ends, ans_begins, ans_ends):
    pre_begin = int(pre_begins)
    pre_end = int(pre_ends)
    ans_begin = int(ans_begins)
    ans_end = int(ans_ends)
    #print(pre_begin, pre_end, ans_begin, ans_end)
    return pre_begin, pre_end, ans_begin, ans_end

In [ ]:
#get new pre array contains only 1 in array 
def linefilter(line_block):
    #print("org error type: ", pre_errortype)
    new_line = [value for value in line_block if value == 1]
    #print("new pre errortype: ", new_pre_errortype)
    return new_line

In [ ]:
 #Note:
    #Predict score:
        # score = inter of ans and pre divide by len of pre
    #Recall score:
        # score = inter of ans and pre divide by len of ans
    #                 b   s   l
    #line sturcture [84][36][150]
def errorline_totalscore(pre_errorline, ans_errorline):
    #get sample size
    sample_size = len(pre_errorline[1])
    #get block size
    lineblock_size = len(pre_errorline)
    total_pre = 0.0
    total_rec = 0.0
    total_sample_pre = 0.0
    total_sample_rec = 0.0
    for sample in range(sample_size):
        sample_totalline_pre = 0.0
        sample_totalline_rec = 0.0
        for lineblock in range(0, lineblock_size, 2):
            #give index blocks value
            pre_begin = pre_errorline[lineblock][sample]
            pre_end = pre_errorline[lineblock+1][sample]
            ans_begin = ans_errorline[lineblock][sample]
            ans_end = ans_errorline[lineblock+1][sample]
            ''' <-------dust switch
            #show error block value
            print("pre_errorline begin[lineblock][sample]: ", lineblock, " ", sample)
            print(pre_begin)
            print("pre_errorline begin[lineblock][sample]: ", lineblock+1, " ", sample)
            print(pre_end)
            print("ans_errorline begin [lineblock][sample]: ", lineblock, " ", sample)
            print(ans_begin)
            print("ans_errorline end[lineblock][sample]: ", lineblock+1, " ", sample)
            print(ans_end)
            #'''
            #get block start/end index
            pre_begin_index = np.where(pre_begin == 1)
            pre_end_index = np.where(pre_end == 1)
            ans_begin_index = np.where(ans_begin == 1)
            ans_end_index = np.where(ans_end == 1)
            ''' <-------dust switch
            #show start/end index
            print("pre_begin_index: ", pre_begin_index)
            print("pre_begin_index type: ", type(pre_begin_index))
            print("pre_end_index: ", pre_end_index)
            print("ans_begin_index: ", ans_begin_index)
            print("ans_end_index: ", ans_end_index)
            #'''
            #give start value
            #Note:
                #use softmax: [0.....150] total == 1
                #begin: index[0]
                #end: index[0]

            #get [value] first and get [value] again to get value
            pre_begin_line, pre_end_line, ans_begin_line, ans_end_line = get_start_end_line(
                                                                                            pre_begin_index, 
                                                                                            pre_end_index, 
                                                                                            ans_begin_index, 
                                                                                            ans_end_index
                                                                                            )
            ''' <-------dust switch
            #show start/end index
            print("pre_begin_line: ", pre_begin_line)
            print("pre_end_line: ", pre_end_line)
            print("ans_begin_line: ", ans_begin_line)
            print("ans_end_line: ", ans_end_line)
            #'''
            ''' <-------dust switch
            #'''
            #make line downgrade for 1
            #Note:
            #before:  0 -1  1  2  3  4
            #after:   0  1 <-----abs  
                        #1  2  3  4
                    #[0, 1, 2, 3, 4, 5]
            pre_begin_line = line_adjustment(pre_begin_line)
            pre_end_line = line_adjustment(pre_end_line)
            ans_begin_line = line_adjustment(ans_begin_line)
            ans_end_line = line_adjustment(ans_end_line)
            ''' <-------dust switch
            print("pre_begin_line: ", pre_begin_line)
            print("pre_end_line: ", pre_end_line)
            print("ans_begin_line: ",ans_begin_line)
            print("ans_end_line: ", ans_end_line)
            #'''
            pre_lineblock = make_lineblock(pre_begin_line, pre_end_line)
            ans_lineblock = make_lineblock(ans_begin_line, ans_end_line)
            #print("pre_lineblock: ", pre_lineblock)
            #print("ans_lineblock: ", ans_lineblock)
            inter = lineblock_intersect(pre_lineblock, ans_lineblock)
            #print("inter: ", inter)
            inter_length = len(inter)
            pre_length = len(pre_lineblock)
            ans_length = len(ans_lineblock)
            #print("inter length: ", inter_length)
            #print("pre_length: ", pre_length)
            #print("ans_length: ", ans_length)
            #count pre_score
            if (inter_length == 0 and pre_length == 0):
                pre_score = 1
            elif (pre_length == 0):
                pre_score = 0
            else :
                pre_score = inter_length/pre_length
            #count rec_score
            if (inter_length == 0 and ans_length == 0):
                rec_score = 1  
                #
            elif (ans_length == 0):
                rec_score = 0
            else:
                rec_score = inter_length/ans_length
            #print("Sample pre_score: ", pre_score)
            #print("Sample rec_score: ", rec_score)
            #cal each sample lineblock score
            sample_totalline_pre += pre_score
            sample_totalline_rec += rec_score
            #print("Sample total_pre: ", sample_pre)
            #print("Sample total_rec: ", sample_rec)
        #cal total sample score
        total_sample_pre += sample_totalline_pre/len(lineblock_size/2)
        total_sample_rec += sample_totalline_rec/len(lineblock_size/2)
    avg_pre = total_sample_pre/sample_size
    avg_rec = total_sample_rec/sample_size
    return avg_pre, avg_rec

In [ ]:
def error_type_F_score(pre_score, rec_score):
    f_one = (2*pre_score*rec_score)/(pre_score + rec_score)
    f_two = (3*pre_score*rec_score)/((2*pre_score) + rec_score)
    f_pointfive = (3*pre_score*rec_score)/(pre_score + (2*rec_score))
    print("F_one: ", f_one)
    print("F_two: ", f_two)
    print("F_pointfive: ", f_pointfive)
    

In [ ]:
def error_line_F_score(pre_score, rec_score):
    f_one = (2*pre_score*rec_score)/(pre_score + rec_score)
    f_two = (3*pre_score*rec_score)/((2*pre_score) + rec_score)
    f_pointfive = (3*pre_score*rec_score)/(pre_score + (2*rec_score))
    print("F_one: ", f_one)
    print("F_two: ", f_two)
    print("F_pointfive: ", f_pointfive)


In [ ]:
pre_errortype, ans_errortype, pre_errorline, ans_errorline = loadmodel(model_name, x_test_model, y_test_mdodel1, y_test_mdodel2)
#print(len(pre_errortype))

'''
work : 
    fix 3 dim prob: 
        out2 only show two dim
        score need three dim: out2: 2D, y_test_loaded_1: 3D
    issue:
        y_test_loaded_1: dim is not expected...
    ==================================================================================
    expect :
    y_test_loaded_0 : [sample = 36][error type = 36] correct v
    out1 :            [sample = 36][error type = 36] correct v
    y_test_loaded_1 : [sample = 36][block = 84][begin and end :line total = 150]
    out2 :            [sample = 36][block = 84][begin and end :line total = 150]
    ==================================================================================
    real : 
            out2 shape            = [36][150]    ------>dim should be [36][84][150]
            y_test_loaded_1 shape = [84][36][150]------>dim should be [36][84][150]
    ==================================================================================
    ref in line work space
'''


In [ ]:
#check all sample error type score
avg_pre, avg_rec, avg_acc = errortype_totalscore(pre_errortype, ans_errortype)
error_type_F_score(avg_pre, avg_rec)

In [ ]:
#check all sample error line score
avg_pre, avg_rec = errorline_totalscore(pre_errorline, ans_errorline)
error_line_F_score(avg_pre, avg_rec)

In [ ]:
#work space...


import numpy as np 
#len(ans_errortype)
sample_size = len(pre_errorline[1])
lineblock_size = len(pre_errorline)
'''
Note:
    pre_errorline[lineblock][sample]
    ans_errorline[lineblock][sample]
Para:
'''


#''' <-------dust switch
print("sample size: ", sample_size)
print("lineblock size: ", lineblock_size)
#'''
total_pre_score = 0.0
total_rec_score = 0.0
for sample in range(sample_size):
    for lineblock in range(0, lineblock_size, 2):
        #give index blocks value
        pre_begin = pre_errorline[lineblock][sample]
        pre_end = pre_errorline[lineblock+1][sample]
        ans_begin = ans_errorline[lineblock][sample]
        ans_end = ans_errorline[lineblock+1][sample]
        
        ''' <-------dust switch
        #show error block value
        print("pre_errorline begin[lineblock][sample]: ", lineblock, " ", sample)
        print(pre_begin)
        print("pre_errorline begin[lineblock][sample]: ", lineblock+1, " ", sample)
        print(pre_end)
        print("ans_errorline begin [lineblock][sample]: ", lineblock, " ", sample)
        print(ans_begin)
        print("ans_errorline end[lineblock][sample]: ", lineblock+1, " ", sample)
        print(ans_end)
        #'''
        #get block start/end index
        pre_begin_index = np.where(pre_begin == 1)
        pre_end_index = np.where(pre_end == 1)
        ans_begin_index = np.where(ans_begin == 1)
        ans_end_index = np.where(ans_end == 1)
        
        ''' <-------dust switch
        #show start/end index
        print("pre_begin_index: ", pre_begin_index)
        print("pre_begin_index type: ", type(pre_begin_index))
        print("pre_end_index: ", pre_end_index)
        print("ans_begin_index: ", ans_begin_index)
        print("ans_end_index: ", ans_end_index)
        #'''
        
        #give start value
        #Note:
            #use softmax: [0.....150] total == 1
            #begin: index[0]
            #end: index[0]
        
        #get [value] first and get [value] again to get value
        #pre_begin_line, pre_end_line, ans_begin_line, ans_end_line = get_start_end_line(pre_begin_index, pre_end_index, ans_begin_index, ans_end_index)
        
        pre_begin_line = pre_begin_index[0]
        #pre_begin_line = pre_begin_line[0]
        pre_end_line = pre_end_index[0]
        #pre_end_line = pre_end_line[0]
        ans_begin_line = ans_begin_index[0]
        ans_begin_line = ans_begin_line[0]
        ans_end_line = ans_end_index[0]
        ans_end_line = ans_end_line[0]
        
        #''' <-------dust switch
        #show start/end index
        print("pre_begin_line: ", pre_begin_line)
        print("pre_end_line: ", pre_end_line)
        print("ans_begin_line: ", ans_begin_line)
        print("ans_end_line: ", ans_end_line)
        #'''
        
        
        
        #make intersection
        
        ''' <-------dust switch
        print("inter_begin: ", inter_begin)
        print("inter_all_begin: ", inter_all_begin)
        #'''
        
        
'''
final decide: find index and build ----> [1 2 3 4 5 6] vs [2 3 4 5 6 7] to make intercestion
'''

In [ ]:
#Note:
    #Predict score:
        # score = inter of ans and pre divide by len of pre
    #Recall score:
        # score = inter of ans and pre divide by len of ans
    #Accuracy score:
        # score = inter_two of ans and pre divide by original len of pre

#case 1:  n  n vs  n  n 
#case 2:  n  n vs -1 -1
#case 3: -1 -1 vs -1 -1
#case 4: -1  n vs  n  n
#case 5: -1  n vs -1  n
pre_begin_line = 5
pre_end_line = 7
ans_begin_line = 3
ans_end_line = 2
pre_begin_line = line_adjustment(pre_begin_line)
pre_end_line = line_adjustment(pre_end_line)
ans_begin_line = line_adjustment(ans_begin_line)
ans_end_line = line_adjustment(ans_end_line)

print("pre_begin_line: ", pre_begin_line)
print("pre_end_line: ", pre_end_line)
print("ans_begin_line: ",ans_begin_line)
print("ans_end_line: ", ans_end_line)
pre_lineblock = make_lineblock(pre_begin_line, pre_end_line)
ans_lineblock = make_lineblock(ans_begin_line, ans_end_line)
print("pre_lineblock: ", pre_lineblock)
print("ans_lineblock: ", ans_lineblock)
inter = lineblock_intersect(pre_lineblock, ans_lineblock)
print("inter: ", inter)
inter_length = len(inter)
pre_length = len(pre_lineblock)
ans_length = len(ans_lineblock)
print("inter length: ", inter_length)
print("pre_length: ", pre_length)
print("ans_length: ", ans_length)
if (inter_length == 0 and pre_length == 0):
    pre_score = 1
    # issue 7.12ver 
elif (pre_length == 0):
    pre_score = 0
else :
    pre_score = inter_length/pre_length
#    
if (inter_length == 0 and ans_length == 0):
    rec_score = 1  
    #
elif (ans_length == 0):
    rec_score = 0
else:
    rec_score = inter_length/ans_length
#print("Sample pre_score: ", pre_score)
total_pre += pre_score
#print("Sample rec_score: ", rec_score)
total_rec += rec_score
#print("Sample total_pre: ", total_pre)
#print("Sample total_rec: ", total_rec)
avg_pre = total_pre/len(pre_begins)
avg_rec = total_rec/len(ans_begins)


In [ ]:
import glob
Input_Path = "Trianing\InputTxt\Split-500"
Input_Path = (glob.glob(Input_Path+"/**/*.txt"))
for i in range(10):
    print(Input_Path[i])

In [ ]:
arr = []
a = ["a","c","d"]
b = ["b","e","f"]
arr.append(a)
arr.append(b)
arr[0][1]